In [244]:
import pandas as pd
from pandasql import sqldf
import pyodbc
import yaml

### Functions

In [245]:
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    
    distance = [[0 for j in range(n+1)] for i in range(m+1)]
    
    for i in range(1, m+1):
        distance[i][0] = i
    for j in range(1, n+1):
        distance[0][j] = j
    
    for j in range(1, n+1):
        for i in range(1, m+1):
            if s[i-1] == t[j-1]:
                cost = 0
            else:
                cost = 1
            distance[i][j] = min(distance[i-1][j] + 1, distance[i][j-1] + 1, distance[i-1][j-1] + cost)
    
    return distance[m][n]


In [246]:
def compare_products(from_db, from_csv, brand):
    name1, name2 = from_db[2], from_csv[1]
    ld = levenshtein_distance("".join(name1.lower().split(brand)),"".join(name2.lower().split(brand)))
    return from_db[0], ld if ld<max(len(name1),len(name2))//3 else None

### Db init

In [247]:
cn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=(localdb)\mssqllocaldb;Database=AffordableSkin;Trusted_Connection=yes')


In [248]:
cr = cn.cursor()

### Get configuration settings

In [249]:
with open(r'config.yaml') as file:
    conf = yaml.load(file, Loader=yaml.FullLoader)
    print(conf)

{'include_files': ['coslovemetics_all_products.csv', 'yeppeuda_all_products.csv', 'olpeo_all_products.csv']}


In [250]:
include_files = conf['include_files']

### Get existing products

In [251]:
all_product_prices_with_ids = []

In [253]:
for file in include_files:
    df = pd.read_csv(file)
    unique_brands = df.brand.unique()
    
    seller = file.split("_")[0]
    sql = f"select max(Id) from Product"
    cr.execute(sql)
    max_id = cr.fetchone()[0] 
    for brand in unique_brands:
        
        sql = "select * from Product where BrandName = ?"
        values = (brand)
        cr.execute(sql,values)
        brand_products = cr.fetchall()

        q = f"""select * from df where brand = '{brand}'"""
        brand_df = sqldf(q, globals())
        for index,row in brand_df.iterrows():
            brand_row = row[4]
            product_id = None
            min_score = 1000
            for product in brand_products:  
                id, score = compare_products(product,row.tolist(),brand)
                if score != None and score<min_score:
                    product_id = id
                    min_score = score
                    
            product_values = [row[0], row[1], 0, row[4]]
            product_prices_values = [row[3], seller, row[2],row[5],row[1]]
            date = row[3]
            if product_id is None:
                
                sql = "insert into Product values (?, ?, ?, ?)"
                
                values = tuple(product_values)
                cr.execute(sql,values)
                cn.commit()
                
                row_add = [max_id+1] + product_prices_values
                all_product_prices_with_ids.append(row_add)
                sql = "insert into ProductPrice values (?, ?, ?, ?, ?,?)"
                values = tuple(row_add)
                cr.execute(sql,values)
                cn.commit()
                max_id+=1
            else:
                sql = "select * from ProductPrice where Date = ? and ProductId = ? and SellerName = ?"
                
                values = (date, product_id, seller)
                cr.execute(sql,values)
                added_products = cr.fetchone()
                row_add = None
                if added_products is None:
                    row_add =  [product_id] + product_prices_values
                    
                else:
                    sql = "insert into Product values (?, ?, ?, ?)"
                    values = tuple(product_values)
                    cr.execute(sql,values)
                    cn.commit()
                    
                    row_add = [max_id+1] + product_prices_values
                    max_id+=1
                
                all_product_prices_with_ids.append(row_add)
                sql = "insert into ProductPrice values (?, ?, ?, ?, ?,?)"
                values = tuple(row_add)
                cr.execute(sql,values)
                cn.commit()

In [254]:
# all_product_prices_with_ids =list(set(map(tuple,all_product_prices_with_ids)))

In [255]:
# for row in all_product_prices_with_ids:
#     sql = "insert into ProductPrice values (?, ?, ?, ?, ?,?)"
#     values = tuple(row)
#     cr.execute(sql,values)
#     cn.commit()

In [256]:
cn.commit()
cr.close()
cn.close()

In [257]:
new_df = pd.DataFrame(all_product_prices_with_ids)

In [258]:
new_df.columns=['productId', 'date','seller','price','link','name']

In [259]:
new_df

,productId,date,seller,price,link,name
0,22194,2023-04-27,coslovemetics,360,https://coslovemetics.mk/product/cp-1-keratin-...,CP-1 Keratin Concentrate Ampoule 10ml
1,22195,2023-04-27,coslovemetics,790,https://coslovemetics.mk/product/cp-1-premium-...,CP-1 Premium Hair Treatment 250ml
2,22196,2023-04-27,coslovemetics,970,https://coslovemetics.mk/product/cp-1-anti-hai...,CP-1 Anti Hair Loss Scalp Infusion Shampoo 250ml
3,22197,2023-04-27,coslovemetics,270,https://coslovemetics.mk/product/cp-1-premium-...,CP-1 Premium Silk Ampoule 20ml
4,22198,2023-04-27,coslovemetics,690,https://coslovemetics.mk/product/cp-1-premium-...,CP-1 Premium Silk Ampoule 150ml
...,...,...,...,...,...,...
1322,23345,2023-04-27,olpeo,2040,https://olpeo.mk/product/yeouth-retinol-moistu...,Yeouth Retinol Moisturizer 60ml
1323,23346,2023-04-27,olpeo,1692,https://olpeo.mk/product/yeouth-retinol-serum-...,Yeouth Retinol Serum 30ml
1324,23347,2023-04-27,olpeo,780,https://olpeo.mk/product/yeouth-retinol-serum-...,Yeouth Retinol Serum 5ml
1325,23348,2023-04-27,olpeo,1387,https://olpeo.mk/product/yeouth-vitamin-c-faci...,Yeouth Vitamin C Facial Cleanser 89ml


In [260]:
new_df.to_csv("total_products_with_ids.csv")